# **Road Line Detection**

Görüntü işleme klasiklerinden olan çizgi tespiti gerçekleştirilecektir.

Görüntüler çizgilerden ve köşelerden oluşur. 

Bu yüzden çizgi tespiti oldukça önemlidir.

Gerçek hayat projesi olarak yolda bulunan şeritlerin tespiti gerçekleştirilecektir.

Otonom araçlarda en temel kurallar aracın gittiği şeridi tespit edebilmesi, şerit değiştirebilmesi ve diğer araçların hangi şeritlerden gittiğini kestirebilmesidir.

Bu nedenle yolda bulunan şeritlerin tespit edilmesi önemli bir konudur.

Yol kenarlarında bulunan çalılar bir sıra halinde diziliyse algoritma tarafından bir şerit olarak algılanabilir.

Bu durum önlenmeye çalışılacaktır.

Yolda uzun bir çizgi tespit edilmeye çalışıldığında etrafta bulunan çalılar, kum tepeleri de tespit edilir, kısa bir çizgi tespit edilmeye çalışıldığında yolda bulunan dönüşler kaçırılır.

Bu trade off'un önlenmesi için parametrelerin uygun bir şekilde ayarlanması gerekmektedir.

In [ ]:
import cv2
import numpy as np

Videoda bulunan önemli bölgelerin alınması için maskeleme işlemi gerçekleştirilir.

In [ ]:
def region_of_interest(image, vertices):
    mask = np.zeros_like(image) #beyaz kısım maskenin seçileceği kısımdır.
    match_mask_color = 255  #255'e beyaza karşılık gelir.
    
    #fillPoly methodu maskeyi oluşturarak kullanılabilir hale getirir.
    cv2.fillPoly(mask, vertices, match_mask_color)
    masked_image = cv2.bitwise_and(image, mask)
    return masked_image

In [ ]:
def draw_lines(image, lines):
    
    image = np.copy(image)
    blank_image = np.zeros((image.shape[0], image.shape[1],3), dtype = np.uint8)
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(blank_image, (x1,y1), (x2,y2), (0,255,0), thickness = 10)
            
    image = cv2.addWeighted(image, 0.8, blank_image, 1, 0.0)
    
    return image

In [ ]:
def process(image):
    height, width = frame.shape[0], frame.shape[1]
    
    region_of_interest_vertices = [(0, height), (width, height), (width, 255)]
    #maskeleme yapmak için kullanılacaktır.
    #projeye göre maskelenecek alan değişiklik göstereceğinden tune edilmesi gerekir.
    
    #converting bgr to grayscacle format
    gray_scale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    #canny methodu ile kenarlar bulunur.
    #parametreler tune edilerek kenarların tespiti gerçekleşir.
    #burada sadece yol üzerindeki şeritlerin değil, yol kenarında bulunan ağaçların, çalıların da kenarlarının tespiti gerçekleşir.
    canny_image = cv2.Canny(gray_scale_image, 250, 120)
    
    #maskeleme fonksiyonu çağrılır.
    cropped_image = region_of_interest(canny_image, np.array([region_of_interest_vertices], np.int32))
    
    #HoughLinesP methodu ile düz çizgiler tespit edilir.
    #minLineLength tune edilmesi gereken bir parametredir, projeye göre değişiklik gösterir.
    #maxLineGap -> gap açıklık demektir. açıklık belirlenerek iki farklı line olduğu tespit edilir.
    lines = cv2.HoughLinesP(cropped_image, rho = 2, theta = np.pi/180, threshold = 200, lines = np.array([]),minLineLength = 150, maxLineGap = 2)
    #print(lines)
    
    image_with_cropped_image_line = draw_lines(image, lines)
    
    return image_with_cropped_image_line

In [ ]:
capture = cv2.VideoCapture(r"C:\Users\ilhan\Desktop\video.mp4")

In [ ]:
#video read
while True:
    
    ret, frame = capture.read()
    
    
    
    if ret == True:
        
        frame = process(frame)

        cv2.imshow("Road Line Detection",frame)
        
        if cv2.waitKey(50) & 0xFF == ord('q'):
            break

    else:
        break
        
capture.release()
cv2.destroyAllWindows()